In [ ]:
import os
import sys
import json
from pathlib import Path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
os.environ['CUDAIDX'] = 'cuda:1'
import random
random.seed(42)

In [ ]:
%env OPENAI_API_KEY=xxx
%env OPENAI_API_BASE=xxx

In [ ]:
import json
from PIL import Image
from IPython.core.display import HTML
from functools import partial
from pathlib import Path

from engine.utils import ProgramGenerator, ProgramInterpreter
from prompts.refcoco import REFCOCO_CURATED_EXAMPLES, REFCOCOG_CURATED_EXAMPLES

In [ ]:
interpreter = ProgramInterpreter(dataset='refcoco')

In [ ]:
def create_prompt(inputs,num_prompts=3,method='random',seed=42,group=0):
    if method=='all':
        prompt_examples = REFCOCO_CURATED_EXAMPLES
    elif method=='random':
        random.seed(seed)
        prompt_examples = random.sample(REFCOCO_CURATED_EXAMPLES,num_prompts)
    else:
        raise NotImplementedError

    prompt_examples = '\n'.join(prompt_examples)
    prompt_examples = f"""Considering the examples provided:\n\n
    {prompt_examples}
    """
    return prompt_examples + "\nInstruction: {instruction}\nProgram:".format(**inputs)

prompter  = partial(create_prompt, method='all')
generator = ProgramGenerator(prompter=prompter,dataset='refcoco')

In [ ]:
from tqdm import tqdm
from PIL import ImageDraw
test_file = os.path.join(Path.home(), 'codes/ExoViP/datasets/refcoco/test.json')
with open(test_file) as jp:
    test = json.load(jp)
eval_pred = 0
eval_cnt = 0

for idx, dct in tqdm(test.items()):
    # eval_cnt += 1
    # if eval_cnt < 5: continue
    
    img_id = dct['img']
    img_path = os.path.join(Path.home(), 'codes/ExoViP/datasets/refcoco/imgs', img_id)
    image = Image.open(img_path)
    
    # draw = ImageDraw.Draw(image)
    # draw.rectangle(dct['box'],outline='red',width=4)
    
    ow, oh = image.size
    # print(ow, oh)
    image.thumbnail((640,640),Image.LANCZOS) # ANTIALIAS
    nw, nh = image.size
    # print(nw, nh)
    wscale = nw/ow
    hscale = nh/oh
    box = dct['box']
    x1 = box[0] * wscale
    y1 = box[1] * hscale
    x2 = box[2] * wscale
    y2 = box[3] * hscale
    label = [x1, y1, x2, y2]
    # label = box
    
    init_state = dict(
        IMAGE=image.convert('RGB'),
        CT_SCORE=0
    )
    instruction = dct['instruction']
    if 'predcit ' not in test[idx]:
        
        results = []
        ct_scores = []
        initial_prompts = []
        initial_prompts.append(generator.generate_prompt(dict(instruction=instruction)))
        porompt_examples = REFCOCO_CURATED_EXAMPLES
        pre_instruct = "Think step by step to carry out the instruction. \
            while taking rejected solutions into account and learning from them. Here are evaluated solutions that were rejected: {rejected_solutions}\n\n \
                Answer the question without making the same mistakes you did with the evaluated rejected solutions. Be simple, Be direct, don't repeat or reply other thing \n\n \
                    Applicable modules include: TAG, LOC, FILTER, RESULT \
                        Following the examples provided:\n\n"
        
        # prog,_ = generator.generate(instruction)
        # # print(prog)
        # result, prog_state = interpreter.execute(prog, init_state, inspect=False)
        # box = prog_state['BOX'][0]
        
        # box, ct_score, cd_score = interpreter.aug_execute(initial_prompts, init_state, instruction, pre_instruct, porompt_examples, task='refcoco')
        # if box == 'NA': box = [0, 0, 0, 0]
        
        try:
            box, ct_score, cd_score = interpreter.aug_execute(initial_prompts, init_state, instruction, pre_instruct, porompt_examples, task='refcoco')
            # prog,_ = generator.generate(dict(instruction=instruction))
            # result, prog_state = interpreter.execute(prog, init_state, inspect=False)
            # box = prog_state['BOX'][0]            
            assert len(box) == 4
        except Exception as e:
            print(e)
            # result = image
            box = [0, 0, 0, 0]
        
        test[idx]['predict'] = box
    else:
        box = test[idx]['predict']
    
    # calculate iou
    label_area = (label[2]-label[0]) * (label[3] - label[1])
    box_area = (box[2]-box[0]) * (box[3] - box[1])
    x1 = max(box[0], label[0])
    x2 = min(box[2], label[2])
    y1 = max(box[1], label[1])
    y2 = min(box[3], label[3])
    intersection = max(0, x2-x1) * max(0, y2-y1)
    iou = intersection / (label_area + box_area - intersection)
    # print(iou)
    eval_pred += iou
    eval_cnt += 1
    
    # # visualize
    # # W,H=image.size
    # draw = ImageDraw.Draw(result)
    # draw.rectangle(label,outline='red',width=4)
    # result.save(f'{idx}.jpg')
    # print(idx, instruction)
    # if eval_cnt > 5:
    #     break
    
    if eval_cnt % 20 == 0:
        print(f'step {eval_cnt} iou: ', round(eval_pred/eval_cnt, 2))
        # break
    
    # ############### eval #####################
    # if result != 'NA' and result != label:
    #     prog, _ = generator.generate(instruction)
    #     print(prog)
    #     reuslt, prog_state, html_str = interpreter.execute(prog, init_state, inspect=True)
    #     print(statement)
    #     print(label)
    #     print(result)
    #     display(HTML(html_str))
    # ###########################################

print('iou: ', eval_pred/len(test.keys()))
result_file = os.path.join(Path.home(), 'codes/ExoViP/results/refcoco/exovip_refcoco.json')
with open(result_file, 'w') as jp:
    json.dump(test, jp)
